# Hexapod Experiments Analysis

Notebook นี้ใช้สำหรับคำนวณค่าที่ต้องกรอกลงในตาราง 3 ตารางตามรูป:

1. **การทดลองที่ 1:** การเคลื่อนที่ของจุดศูนย์กลางมวล (ใช้ไฟล์ `base_link_*.csv`)
2. **การทดลองที่ 2:** ความเร็วสูงสุดของหุ่นยนต์ Hexapod (ใช้ไฟล์ `base_link_*.csv`)
3. **การทดลองที่ 3:** mean Absolute Position Tracking Error ของข้อต่อและปลายเท้า (ใช้ไฟล์ `leg_*.csv`)

> แนะนำให้เซฟไฟล์ `.csv` ทั้งหมดให้อยู่ในโฟลเดอร์เดียวกับโน้ตบุ๊กนี้ก่อนรัน


In [30]:
import pandas as pd
import numpy as np
import math
import glob
import os

# ปรับ path ถ้าไฟล์อยู่คนละโฟลเดอร์
path = os.listdir('/home/ambushee/hexapod_logs/')
print("Files in hexapod_logs:", path)
BASE_PATTERN = "/home/ambushee/hexapod_logs/base_link_*.csv"
LEG_PATTERN = "/home/ambushee/hexapod_logs/leg_[1-6]_*.csv"

base_files = sorted(glob.glob(BASE_PATTERN))
leg_files = sorted(glob.glob(LEG_PATTERN))

print("base_link files:", base_files)
print("leg files:", leg_files)

if not base_files:
    print("\n⚠️ ไม่พบไฟล์ base_link_*.csv - การทดลองที่ 1 และ 2 จะคำนวณไม่ได้ จนกว่าจะมีไฟล์นี้")
if len(leg_files) == 0:
    print("\n⚠️ ไม่พบไฟล์ leg_*.csv - การทดลองที่ 3 จะคำนวณไม่ได้")


Files in hexapod_logs: ['base_link_20251128_013017.csv', 'trial1', 'leg_2_20251128_013017.csv', 'leg_3_20251128_013017.csv', 'leg_5_20251128_013017.csv', 'leg_4_20251128_013017.csv', 'trial2', 'leg_1_20251128_013017.csv', 'leg_6_20251128_013017.csv']
base_link files: ['/home/ambushee/hexapod_logs/base_link_20251128_013017.csv']
leg files: ['/home/ambushee/hexapod_logs/leg_1_20251128_013017.csv', '/home/ambushee/hexapod_logs/leg_2_20251128_013017.csv', '/home/ambushee/hexapod_logs/leg_3_20251128_013017.csv', '/home/ambushee/hexapod_logs/leg_4_20251128_013017.csv', '/home/ambushee/hexapod_logs/leg_5_20251128_013017.csv', '/home/ambushee/hexapod_logs/leg_6_20251128_013017.csv']


In [31]:
# -------------------------------
# การทดลองที่ 1
# จุดศูนย์กลางมวลเคลื่อนที่จากจุดเริ่มต้นเท่าไร
# -------------------------------

def compute_exp1_from_base(df):
    if df.empty:
        raise ValueError("DataFrame ว่าง (ไม่มีข้อมูลจาก base_link)")
    # ใช้ตำแหน่งแกน x,y แถวแรกและแถวสุดท้าย
    x_start, y_start = df.loc[df.index[0], ["pos_x", "pos_y"]]
    x_end,   y_end   = df.loc[df.index[-1], ["pos_x", "pos_y"]]
    dx, dy = x_end - x_start, y_end - y_start

    # มุมทิศทางของเวกเตอร์การเคลื่อนที่ (rad)
    angle_rad = math.atan2(dy, dx)
    displacement = math.hypot(dx, dy)

    return {
        "x_start": x_start,
        "y_start": y_start,
        "x_end": x_end,
        "y_end": y_end,
        "angle_rad": angle_rad,
        "displacement": displacement,
    }

if base_files:
    base_df = pd.read_csv(base_files[0])
    print("Base shape:", base_df.shape)

    if base_df.empty:
        print("\n⚠️ base_link.csv ไม่มีข้อมูล (มีแต่ header) กรุณาเช็คขั้นตอน log ข้อมูลใน ROS/Controller อีกครั้ง")
    else:
        exp1_result = compute_exp1_from_base(base_df)
        exp1_table = pd.DataFrame(
            [
                [1,
                 exp1_result["x_start"],
                 exp1_result["y_start"],
                 exp1_result["x_end"],
                 exp1_result["y_end"],
                 exp1_result["angle_rad"]]
            ],
            columns=[
                "trial",
                "x_start (m)",
                "y_start (m)",
                "x_end (m)",
                "y_end (m)",
                "angle_change (rad)",
            ],
        )
        print("ผลสำหรับการทดลองครั้งที่ 1 (ใช้กรอกตารางบนสุด):")
        display(exp1_table)
else:
    print("ไม่มี base_link file สำหรับการทดลองที่ 1")


Base shape: (4101, 8)
ผลสำหรับการทดลองครั้งที่ 1 (ใช้กรอกตารางบนสุด):


,trial,x_start (m),y_start (m),x_end (m),y_end (m),angle_change (rad)
0,1,0.0,0.0,0.0,0.0,0.0


In [32]:
# -------------------------------
# การทดลองที่ 2
# ความเร็วสูงสุดของหุ่นยนต์ Hexapod
# -------------------------------

def compute_max_speed(df):
    if df.empty:
        raise ValueError("DataFrame ว่าง (ไม่มีข้อมูลจาก base_link)")
    # ถ้ามีคอลัมน์ vel_x, vel_y, vel_z ให้ใช้คำนวณความเร็วรวม
    if all(col in df.columns for col in ["vel_x", "vel_y", "vel_z"]):
        speed = np.sqrt(df["vel_x"]**2 + df["vel_y"]**2 + df["vel_z"]**2)
    else:
        raise ValueError("ไม่พบคอลัมน์ vel_x, vel_y, vel_z ใน base_link.csv")
    return speed.max(), speed.idxmax()

if base_files:
    base_df = pd.read_csv(base_files[0])
    if base_df.empty:
        print("\n⚠️ base_link.csv ไม่มีข้อมูล (มีแต่ header) การทดลองที่ 2 จึงยังคำนวณไม่ได้")
    else:
        max_speed, idx = compute_max_speed(base_df)
        t_peak = base_df.loc[idx, "timestamp"] if "timestamp" in base_df.columns else None

        print("ความเร็วสูงสุด (m/s):", max_speed)
        if t_peak is not None:
            print("เวลา (timestamp) ที่ความเร็วสูงสุด:", t_peak)

        # สร้าง DataFrame 1 แถว สำหรับกรอกตารางกลาง
        exp2_table = pd.DataFrame(
            [[1, max_speed]],
            columns=["trial", "max_speed (m/s)"],
        )
        display(exp2_table)
else:
    print("ไม่มี base_link file สำหรับการทดลองที่ 2")


ความเร็วสูงสุด (m/s): 0.0
เวลา (timestamp) ที่ความเร็วสูงสุด: 1764268217.9447167


,trial,max_speed (m/s)
0,1,0.0


In [33]:
# -------------------------------
# การทดลองที่ 3
# mean Absolute Position Tracking Error
# -------------------------------

def compute_leg_errors(df):
    required_joint_cols = [
        "joint_hip_pos", "joint_knee_pos", "joint_ankle_pos",
        "joint_hip_target", "joint_knee_target", "joint_ankle_target",
    ]
    required_ee_cols = [
        "ee_pos_x", "ee_pos_y", "ee_pos_z",
        "ee_target_x", "ee_target_y", "ee_target_z",
    ]
    for col in required_joint_cols + required_ee_cols:
        if col not in df.columns:
            raise ValueError(f"ไม่พบคอลัมน์ {col} ในข้อมูลขา")

    hip_err   = (df["joint_hip_pos"]   - df["joint_hip_target"]).abs()
    knee_err  = (df["joint_knee_pos"]  - df["joint_knee_target"]).abs()
    ankle_err = (df["joint_ankle_pos"] - df["joint_ankle_target"]).abs()

    ee_x_err = (df["ee_pos_x"] - df["ee_target_x"]).abs()
    ee_y_err = (df["ee_pos_y"] - df["ee_target_y"]).abs()
    ee_z_err = (df["ee_pos_z"] - df["ee_target_z"]).abs()

    return {
        "hip": hip_err.mean(),
        "knee": knee_err.mean(),
        "ankle": ankle_err.mean(),
        "ee_x": ee_x_err.mean(),
        "ee_y": ee_y_err.mean(),
        "ee_z": ee_z_err.mean(),
    }

if leg_files:
    per_leg_errors = []
    for path in leg_files:
        df_leg = pd.read_csv(path)
        errs = compute_leg_errors(df_leg)
        errs["leg_file"] = os.path.basename(path)
        per_leg_errors.append(errs)

    leg_error_df = pd.DataFrame(per_leg_errors)
    display(leg_error_df)

    # เฉลี่ย error ของทุกขา (global mean absolute tracking error)
    mean_errors = {
        "hip":  leg_error_df["hip"].mean(),
        "knee": leg_error_df["knee"].mean(),
        "ankle": leg_error_df["ankle"].mean(),
        "ee_x": leg_error_df["ee_x"].mean(),
        "ee_y": leg_error_df["ee_y"].mean(),
        "ee_z": leg_error_df["ee_z"].mean(),
    }

    exp3_table = pd.DataFrame(
        [
            [1,
             mean_errors["hip"],
             mean_errors["knee"],
             mean_errors["ankle"],
             mean_errors["ee_x"],
             mean_errors["ee_y"],
             mean_errors["ee_z"],
            ]
        ],
        columns=[
            "trial",
            "hip_link_error (rad)",
            "knee_link_error (rad)",
            "ankle_link_error (rad)",
            "ee_x_error (m)",
            "ee_y_error (m)",
            "ee_z_error (m)",
        ],
    )

    print("เฉลี่ย error ในทุกขา ใช้กรอกตารางล่างสุดแถวการทดลองครั้งที่ 1:")
    display(exp3_table)
else:
    print("ไม่มี leg_*.csv สำหรับการทดลองที่ 3")


,hip,knee,ankle,ee_x,ee_y,ee_z,leg_file
0,0.129737,0.223255,0.151491,0.025370,0.032613,0.014482,leg_1_20251128_013017.csv
1,0.172283,0.302964,0.218494,0.039736,0.031709,0.012972,leg_2_20251128_013017.csv
2,0.151418,0.244374,0.205427,0.033689,0.022861,0.016604,leg_3_20251128_013017.csv
3,0.161443,0.243963,0.193660,0.031716,0.028578,0.017028,leg_4_20251128_013017.csv
4,0.143960,0.244314,0.189429,0.048563,0.034928,0.016739,leg_5_20251128_013017.csv
5,0.139217,0.310828,0.216921,0.032639,0.030333,0.017100,leg_6_20251128_013017.csv


เฉลี่ย error ในทุกขา ใช้กรอกตารางล่างสุดแถวการทดลองครั้งที่ 1:


,trial,hip_link_error (rad),knee_link_error (rad),ankle_link_error (rad),ee_x_error (m),ee_y_error (m),ee_z_error (m)
0,1,0.149676,0.261616,0.195904,0.035286,0.03017,0.015821


In [34]:
# -------------------------------
# การทดลองที่ 4
# mean Absolute Velocity Tracking Error
# -------------------------------

def compute_leg_velocity_errors(df):
    required_joint_vel_cols = [
        "joint_hip_vel", "joint_knee_vel", "joint_ankle_vel",
        "joint_hip_target_vel", "joint_knee_target_vel", "joint_ankle_target_vel",
    ]
    for col in required_joint_vel_cols:
        if col not in df.columns:
            raise ValueError(f"ไม่พบคอลัมน์ {col} ในข้อมูลขา")

    hip_vel_err   = (df["joint_hip_vel"]   - df["joint_hip_target_vel"]).abs()
    knee_vel_err  = (df["joint_knee_vel"]  - df["joint_knee_target_vel"]).abs()
    ankle_vel_err = (df["joint_ankle_vel"] - df["joint_ankle_target_vel"]).abs()

    return {
        "hip_vel": hip_vel_err.mean(),
        "knee_vel": knee_vel_err.mean(),
        "ankle_vel": ankle_vel_err.mean(),
    }   
if leg_files:
    per_leg_vel_errors = []
    for path in leg_files:
        df_leg = pd.read_csv(path)
        vel_errs = compute_leg_velocity_errors(df_leg)
        vel_errs["leg_file"] = os.path.basename(path)
        per_leg_vel_errors.append(vel_errs)

    leg_vel_error_df = pd.DataFrame(per_leg_vel_errors)
    display(leg_vel_error_df)

    # เฉลี่ย velocity error ของทุกขา (global mean absolute velocity tracking error)
    mean_vel_errors = {
        "hip_vel":  leg_vel_error_df["hip_vel"].mean(),
        "knee_vel": leg_vel_error_df["knee_vel"].mean(),
        "ankle_vel": leg_vel_error_df["ankle_vel"].mean(),
    }

    exp4_table = pd.DataFrame(
        [
            [1,
             mean_vel_errors["hip_vel"],
             mean_vel_errors["knee_vel"],
             mean_vel_errors["ankle_vel"],
            ]
        ],
        columns=[
            "trial",
            "hip_link_velocity_error (rad/s)",
            "knee_link_velocity_error (rad/s)",
            "ankle_link_velocity_error (rad/s)",
        ],
    )

    print("เฉลี่ย velocity error ในทุกขา ใช้กรอกตารางล่างสุดแถวการทดลองครั้งที่ 1:")
    display(exp4_table)
else:
    print("ไม่มี leg_*.csv สำหรับการทดลองที่ 4")
    


,hip_vel,knee_vel,ankle_vel,leg_file
0,5.367936,5.536329,5.709104,leg_1_20251128_013017.csv
1,5.617346,5.346562,5.650172,leg_2_20251128_013017.csv
2,5.766862,5.416394,5.979220,leg_3_20251128_013017.csv
3,5.760092,5.268568,5.714627,leg_4_20251128_013017.csv
4,5.876004,5.598664,6.049371,leg_5_20251128_013017.csv
5,5.453725,5.196942,5.592244,leg_6_20251128_013017.csv


เฉลี่ย velocity error ในทุกขา ใช้กรอกตารางล่างสุดแถวการทดลองครั้งที่ 1:


,trial,hip_link_velocity_error (rad/s),knee_link_velocity_error (rad/s),ankle_link_velocity_error (rad/s)
0,1,5.640328,5.39391,5.782456


In [ ]:
# -------------------------------
# การทดลองที่ 5
# mean Absolute End Effector Velocity Tracking Error
# -------------------------------

def compute_ee_velocity_errors(df):
    required_ee_vel_cols = [
        "ee_vel_x", "ee_vel_y", "ee_vel_z",
        "ee_target_vel_x", "ee_target_vel_y", "ee_target_vel_z",
    ]
    for col in required_ee_vel_cols:
        if col not in df.columns:
            raise ValueError(f"ไม่พบคอลัมน์ {col} ในข้อมูลขา")

    ee_vel_x_err = (df["ee_vel_x"] - df["ee_target_vel_x"]).abs()
    ee_vel_y_err = (df["ee_vel_y"] - df["ee_target_vel_y"]).abs()
    ee_vel_z_err = (df["ee_vel_z"] - df["ee_target_vel_z"]).abs()

    return {
        "ee_vel_x": ee_vel_x_err.mean(),
        "ee_vel_y": ee_vel_y_err.mean(),
        "ee_vel_z": ee_vel_z_err.mean(),
    }

if leg_files:
    per_leg_ee_vel_errors = []
    for path in leg_files:
        df_leg = pd.read_csv(path)
        ee_vel_errs = compute_ee_velocity_errors(df_leg)
        ee_vel_errs["leg_file"] = os.path.basename(path)
        per_leg_ee_vel_errors.append(ee_vel_errs)

    leg_ee_vel_error_df = pd.DataFrame(per_leg_ee_vel_errors)
    display(leg_ee_vel_error_df)

    # เฉลี่ย end effector velocity error ของทุกขา
    mean_ee_vel_errors = {
        "ee_vel_x": leg_ee_vel_error_df["ee_vel_x"].mean(),
        "ee_vel_y": leg_ee_vel_error_df["ee_vel_y"].mean(),
        "ee_vel_z": leg_ee_vel_error_df["ee_vel_z"].mean(),
    }

    exp5_table = pd.DataFrame(
        [
            [1,
             mean_ee_vel_errors["ee_vel_x"],
             mean_ee_vel_errors["ee_vel_y"],
             mean_ee_vel_errors["ee_vel_z"],
            ]
        ],
        columns=[
            "trial",
            "ee_vel_x_error (m/s)",
            "ee_vel_y_error (m/s)",
            "ee_vel_z_error (m/s)",
        ],
    )

    print("เฉลี่ย end effector velocity error ในทุกขา:")
    display(exp5_table)
else:
    print("ไม่มี leg_*.csv สำหรับการทดลองที่ 5")
    ## eiei

,ee_vel_x,ee_vel_y,ee_vel_z,leg_file
0,2.483986,1.845352,1.110615,leg_1_20251128_013017.csv
1,1.700782,2.620135,0.820428,leg_2_20251128_013017.csv
2,2.373054,2.692824,1.479886,leg_3_20251128_013017.csv
3,3.029224,1.793194,1.239988,leg_4_20251128_013017.csv
4,1.578349,1.909661,0.945887,leg_5_20251128_013017.csv
5,2.116539,2.780309,1.572172,leg_6_20251128_013017.csv


เฉลี่ย end effector velocity error ในทุกขา:


,trial,ee_vel_x_error (m/s),ee_vel_y_error (m/s),ee_vel_z_error (m/s)
0,1,2.213656,2.273579,1.194829
